In [22]:
!pip install --upgrade gensim
!pip install tqdm

In [23]:
import pickle
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore
from tqdm import tqdm

In [24]:
with open('pickle/df_proc.p', 'rb') as file:
    df = pickle.load(file)
df.head()

,text
0,programa treinamento jornalismo agroindústria ...
1,ofício enviado juiz sergio moro feira presiden...
2,ex governador rio sérgio cabral transferido co...
3,aplausos vaias juiz federal sergio moro respon...
4,ex executivo empreiteira odebrecht afirmou aco...


In [25]:
tokenized_docs = [doc.lower().split() for doc in df['text']]

In [26]:
id2word = corpora.Dictionary(tokenized_docs)
corpus = [id2word.doc2bow(text) for text in tokenized_docs]

In [27]:
def compute_coherence_values(corpus, dictionary, texts, range_lda): # meu
    coherence_values = []
    model_list = []
    for num_topics in tqdm(range_lda):
        model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, random_state=42)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())
    return model_list, coherence_values

In [ ]:
limit=151; start=2; step=1;
range_lda = range(start, limit, step)
model_list, coherence_values = compute_coherence_values(corpus=corpus, dictionary=id2word, texts=tokenized_docs, range_lda=range_lda)

 42%|████▏     | 62/149 [11:05<21:07, 14.57s/it]

In [ ]:
with open('model_list.p', 'wb') as file:
    pickle.dump(model_list, file)

with open('coherence_values.p', 'wb') as file:
    pickle.dump(coherence_values, file)